In [2]:
import requests
from bs4 import BeautifulSoup
import dateparser
import pandas as pd
from tqdm import tqdm
import unicodedata
from pathlib import Path

In [2]:
def get_soup_parser(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, "html.parser")

In [19]:
def get_mali_jet_page_list_of_articles(num_page):
    soup = get_soup_parser(url=f"https://malijet.com/a_la_une_du_mali/?page={num_page}")
    articles = soup.find("div", id="v_container").find_all("div", class_="card")
    titles, source_papers, dates, links = [], [], [], []
    print('Getting list of articles...')
    for article in tqdm(articles[:-1]):
        header = article.find("div", class_="card-header")
        link = header.find("a", href=True)
        title = None if not header else unicodedata.normalize("NFKD", header.text.strip().split("\n")[-1])
        infos = article.find("div", class_="card-body")
        infos = None if not infos else infos.text.strip().split("\n")
        
        titles.append(title)
        source_papers.append(None if not infos else infos[0])
        dates.append(None if not infos or not dateparser.parse(infos[1]) else dateparser.parse(infos[1]).date())
        links.append(unicodedata.normalize("NFKD", link['href']))
        # print("*"*100)
    return pd.DataFrame({"title": titles, "source_paper": source_papers, "date": dates, "link": links})

## Extract info from one article


In [16]:
def fetch_article_content(article_link):
    soup = get_soup_parser(url=article_link)
    
    # get content
    content = " ".join(paragraph.text for paragraph in soup.find_all("div", dir="auto") if not paragraph.text.isspace())
    
    # TODO : We must implement a way to parse the article's author and return it as a tuple with "content"
    # author = ""
    
    if content != '':
        return content
    else:
        large_paragraphs = soup.find("div", class_="card-header").text.split('Date : ')[1].split('À lire aussi \n\n\n')
        if len(large_paragraphs) > 1: 
            large_paragraphs[1] = large_paragraphs[1].split('\n\n\n\n')[-1] # take the text after "A lire aussi"
        final_content = " ".join(large_paragraphs)
        return unicodedata.normalize("NFKD", " ".join(final_content.split('\n')[1:])).strip().replace("     ", " ")

In [47]:
new_article_link = "https://malijet.com/a_la_une_du_mali/290531-industrie--le-president-assimi-goita-a-recu-l’ancien-footballeur.html"

In [48]:
fetch_article_content(new_article_link)

'Le Président de la Transition, Son Excellence le Colonel Assimi GOÏTA, Chef de l’État, a reçu en audience, ce mardi 07 mai 2024, Seydou KEÏTA, ancien footballeur international et entrepreneur visionnaire, en prélude de la finalisation de son complexe industriel à Sanankoroba, dans le Cercle de Kati. Cette rencontre, qui s’est tenue en présence du ministre de l’Industrie et du Commerce, marque un jalon important dans le parcours de M. KEÏTA et pour le développement économique du Mali. Seydou KEÏTA, qui a entamé ce projet ambitieux en 2021, a transformé son rêve en réalité, illustrant parfaitement la transition réussie d’une carrière sportive à celle d’entrepreneur engagé. Le complexe industriel, résultat d’années d’efforts et d’investissements considérables, est désormais prêt à démarrer ses opérations, promettant de devenir un moteur de croissance pour la région et pour le Mali tout entier. Au cours de cette audience, Seydou KEÏTA a exprimé sa gratitude envers les autorités maliennes 

In [5]:
new_article_link = "https://malijet.com/a_la_une_du_mali/290746-les-coulisses-du-dim -le-pupitre-presidentiel.html"

In [6]:
fetch_article_content(new_article_link)

'À l’ouverture comme à la clôture des travaux du dernier round du DIM, deux pupitres ont été installés aux côtés du présidium : celui de gauche pour les autres intervenants et celui d’à droite réservé exclusivement au maître de céans, Colonel Assimi Goïta. Jusqu’à la fin de la cérémonie, il n’y a eu aucune méprise. Qui est fou ? Magma et la «peur» des femmes Entré dans la salle Wa Kamissoko, où siégeait la “Commission économie et développement durable”, Magma Gabriel Konaté, Président de la Commission culture du Cnt, a été surpris par la présence massive des femmes. Il est resté débout quelques minutes avant qu’un délégué ne l’invite à s’asseoir à ses côtés. Magma a tenu, avec l’humour qu’on lui connaît, à justifier son apparent embarras. «Eh, je vais m’asseoir ici à côté, dans un coin de la salle....car j’ai peur quand il y a beaucoup de femmes...». C’était peu avant la reprise des travaux, et il est effectivement resté dans  son coin jusqu

## Second part: Parsing using date

In [20]:
begin_date = "2024-05-13"
end_date = "2024-05-14" #today
# parse them
begin_date = dateparser.parse(begin_date).date()
end_date = dateparser.parse(end_date).date()

In [21]:
page_number = 1
articles_to_fetch_df = pd.DataFrame(columns=["title", "source_paper", "date", "link"])
contents = []
current_date = end_date
while begin_date <= current_date:
    print(f"fetching article from page {page_number} ...")
    articles_to_fetch_df = pd.concat([articles_to_fetch_df, get_mali_jet_page_list_of_articles(page_number)])
    page_number+=1
    current_date = articles_to_fetch_df.date.min()

articles_to_fetch_df.query("date >= @begin_date and date <= @end_date")

fetching article from page 1 ...
Getting list of articles...


100%|██████████| 20/20 [00:00<00:00, 463.40it/s]


,title,source_paper,date,link
6,Les coulisses du DIM : Le pupitre présidentiel,Le Challenger,2024-05-14,https://malijet.com/a_la_une_du_mali/290746-le...
7,Dialogue Inter-maliens pour la paix et la réc...,Le Challenger,2024-05-14,https://malijet.com/a_la_une_du_mali/290745-di...
8,Dialogue inter-malien : des recommandations on...,Malijet,2024-05-13,https://malijet.com/a_la_une_du_mali/290715-di...
9,Fin du Dialogue inter-Maliens : remise du rapp...,Présidence,2024-05-13,https://malijet.com/a_la_une_du_mali/290712-fi...
10,Mali : le dialogue inter-maliens débouche sur...,Anadolu,2024-05-13,https://malijet.com/a_la_une_du_mali/290707-ma...
11,Métier Postal : Des acteurs postaux à l’éco...,Malijet,2024-05-13,https://malijet.com/a_la_une_du_mali/290706-me...
12,"Salon des médias : le Maroc et la Chine, pays...",Malijet,2024-05-13,https://malijet.com/a_la_une_du_mali/290705-sa...
13,Recommandations phares du dialogue inter-maliens,Le Républicain,2024-05-13,https://malijet.com/a_la_une_du_mali/290700-re...
14,Pour tout savoir sur les recommandations du di...,Malijet,2024-05-13,https://malijet.com/a_la_une_du_mali/290699-po...
15,EMGA : Les adieux du Commandant de l’EUTM,Fama,2024-05-13,https://malijet.com/a_la_une_du_mali/290695-em...


In [10]:
CSV_DIR = Path().resolve() / 'data' / 'malijet' / 'source.csv'
CSV_DIR

PosixPath('/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv')

In [11]:
subset_fetching_articles_df = articles_to_fetch_df.query("date >= @begin_date and date <= @end_date").copy()
article_contents, new_titles = [], []
existing_article_titles = pd.read_csv(CSV_DIR, sep='\t').title.tolist()
for _, row in tqdm(subset_fetching_articles_df.iterrows(), total=subset_fetching_articles_df.shape[0]):
    if row.title not in existing_article_titles:
        new_titles.append(row.title)
        article_contents.append(fetch_article_content(row.link))
if article_contents:
    print("New articles found, writing article contents to file...")
    subset_fetching_articles_df.query("title in @new_titles").assign(content=article_contents).to_csv(CSV_DIR, mode='a', sep='\t', index=False, header=False)
else:
    print("No new articles found, skipping...")

100%|██████████| 32/32 [00:00<00:00, 14641.40it/s]

No new articles found, skipping...


In [12]:
from pathlib import Path
import pandas as pd
ARTICLE_DIRECTORY_PATH = (
        Path().resolve().parents[1] / "new_test" / "data" / "articles" / "malijet" / "2024" / "01" / "01.csv"
    )
ARTICLE_DIRECTORY_PATH

PosixPath('/home/bouba/Workspace/new_test/data/articles/malijet/2024/01/01.csv')

In [17]:
df = pd.read_csv(
    ARTICLE_DIRECTORY_PATH, delimiter="\t", encoding = "latin1"
)
df

,title,source_paper,date,link,content
0,Mali: le Col Assimi GOITA annonce la creÌatio...,Malijet,2024-01-01,https://malijet.com/a_la_une_du_mali/286070-ma...,C'eÌtait lors de son discours du Nouvel An. ...
1,Ronde des airs de santeÌ aÌ la veille du nou...,Malijet,2024-01-01,https://malijet.com/a_la_une_du_mali/286069-ro...,"Au seuil du nouvel an 2024, Madame le Ministre..."
2,A lâoreÌe du nouvel: Le ministre de la Sant...,Malijet,2024-01-01,https://malijet.com/a_la_une_du_mali/286056-a-...,Câest deÌsormais une tradition pour le min...


In [14]:
pd.read_table(ARTICLE_DIRECTORY_PATH)

,title,source_paper,date,link,content
0,Mali: le Col Assimi GOITA annonce la création...,Malijet,2024-01-01,https://malijet.com/a_la_une_du_mali/286070-ma...,C'était lors de son discours du Nouvel An. I...
1,Ronde des airs de santé à la veille du nouve...,Malijet,2024-01-01,https://malijet.com/a_la_une_du_mali/286069-ro...,"Au seuil du nouvel an 2024, Madame le Ministre..."
2,A l’orée du nouvel: Le ministre de la Santé ...,Malijet,2024-01-01,https://malijet.com/a_la_une_du_mali/286056-a-...,C’est désormais une tradition pour le minist...
